PCLab#5 - Group 2 - Emanuele Sala, Luca Soleri, Fabio Stefana

<div style="border: 4px solid #007bff; padding: 10px; background-color: #e9f5ff; border-radius: 5px;">
    <h1 style="color: #007bff;">Importing libraries and Dataset</h1>
</div>


In [1]:
import pandas as pd
import os

In [16]:
directory = "data/sigwatch_data"
df_list = []

for file in os.listdir(directory):
    if file.endswith(".dta"):
        file_path = os.path.join(directory, file)
        temp_df = pd.read_stata(file_path)
        df_list.append(temp_df)
        
df = pd.concat(df_list, ignore_index=True)

<div style="border: 4px solid #007bff; padding: 10px; background-color: #e9f5ff; border-radius: 5px;">
    <h1 style="color: #007bff;">Preliminary data exploration</h1>
</div>

In [17]:
# With this filter we keep only the banks
df = df[df["corp_industry_sector1"] == "Finance"]

In [18]:
# And with this filter we only keep the countries form the US, UK or EU
countries = ['Austria',
             'US',
             'Denmark',
             'UK',
             'Germany',
             'Luxembourg',
             'France',
             'Italy',
             'Netherlands',
             'Belgium',
             'Sweden',
             'Spain',
             'Ireland',
             'Portugal',
             'Poland',
             'Finland',
             'USA',
             'Croatia',
             'Bulgaria',
             'Montenegro',
             'Bosnia and Herzegovina']

df = df[df['country_corp'].isin(countries)]

In [19]:
# We count the unique number of ud_archive as some have more than one row but still count as one isngle campaing
n_of_campaigns = len(list(df["uid_archive"].unique()))
print(f"There are {n_of_campaigns} unique campaigns for US UK and EU banks")

There are 2370 unique campaigns for US UK and EU banks


In [20]:
list_of_ngo_columns = []
for i in range(5):
    i = i+1
    ngo_column_number = f"ngo_name{i}"
    ngo_col = list(df[ngo_column_number])
    list_of_ngo_columns += ngo_col
unique_ngos = list(set(list_of_ngo_columns))

# we do -1 because we have to account for the null value
print(f"There are {len(unique_ngos) - 1} unique NGO organizations involved in this dataset")

There are 932 unique NGO organizations involved in this dataset


In [7]:
# Numero di aziende targettate

In [21]:
columns_we_want = ["uid_archive", 
                   "date", 
                   "company",
                   'country_corp', # Country of the Company
                   'corp_industry_sector1', # Industry of the company
                   'company_parent',
                   'company_parent_country',
                   "sentiment",
                   'issue_name1',
                   'issue_name2',
                   'issue_name3']

# Reddit Analysis

In [86]:
import praw
import pandas as pd
from datetime import datetime, timedelta

# Reddit API credentials
client_id = "RQswmwIqjV3DmpuBke34QQ"
client_secret = "EhyqqRfPu3x4EDVnsGTMD12y0xFbBQ"
user_agent = "finance_scrape (by u/Kashiko_02)"

# Set up the Reddit client
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Sample data from your dataset
#campaigns = [df['company'], df['ngo_name1'], df['date']]

df_1 = df.copy()
#keep only first few words of the report
df_1['report'] = df_1['report'].str[:100]

columns_to_keep = ['date', 'company', 'ngo_name1', 'ngo_name2', 'ngo_name3', 'ngo_name4', 'ngo_name5','report']
campaign = df_1.reindex(columns=columns_to_keep)
campaign = campaign.iloc[:50]


# Data collection
reddit_data = []

for index, campaign in campaign.iterrows():
    ngos = [campaign['ngo_name1'], campaign['ngo_name2'], campaign['ngo_name3'], 
        campaign['ngo_name4'], campaign['ngo_name5']]
    
    campaign_date = datetime.strptime(campaign["date"], "%Y-%m-%d")
    campaign_year = campaign_date.year

    # Filter out empty NGO names and join them with spaces
    ngos = "; ".join([ngo for ngo in ngos if ngo])
    search_query = f"{campaign['company']} {campaign['report']} {ngos}".strip()  
    #    print(search_query)
    
    
    search_limit = 100
    
    # Perform the search
    submissions = reddit.subreddit("finance").search(search_query, limit=search_limit)
    
    
    # Convert campaign date to timestamp range
    date_before = campaign_date + timedelta(days=365)
    date_after = campaign_date - timedelta(days=100)

    # Filter and collect data
    for submission in submissions:
        # Convert Reddit post date
        post_date = datetime.utcfromtimestamp(submission.created_utc)
        post_date = datetime.strptime(str(post_date), "%Y-%m-%d %H:%M:%S")
        post_date = post_date.strftime("%Y-%m-%d")
        post_date = datetime.strptime(post_date, "%Y-%m-%d")
        
        
        #print(date_after,post_date,date_before)
        
        # Check if post falls within the date range
        if date_after <= post_date <= date_before:
            reddit_data.append({
                "company": campaign["company"],
                "NGO": ngos,
                "date": campaign["date"],
                "title": submission.title,
                "text": submission.selftext,
                "created_utc": submission.created_utc,
                "url": submission.url,
            #    "subreddit": submission.subreddit.display_name
            })
            

# Save to a DataFrame
df_reddit = pd.DataFrame(reddit_data)


# The next steps would be to clean the text for sentiment analysis and apply a sentiment model.

df_reddit.to_csv("reddit_data.csv", index=False)

In [ ]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import time


# Reddit API credentials
client_id = "RQswmwIqjV3DmpuBke34QQ"
client_secret = "EhyqqRfPu3x4EDVnsGTMD12y0xFbBQ"
user_agent = "finance_scrape (by u/Kashiko_02)"

# Set up the Reddit client
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Sample data from your dataset
df_1 = df.copy()
# Keep only the first few words of the report (to limit the query size)
df_1['report'] = df_1['report'].str[:100]

columns_to_keep = ['date', 'company', 'ngo_name1', 'ngo_name2', 'ngo_name3', 'ngo_name4', 'ngo_name5', 'report']
campaign = df_1.reindex(columns=columns_to_keep)
campaign = campaign.iloc[5000:5200]


# Data collection
reddit_data = []

for index, campaign in campaign.iterrows():
    if index % 50 == 0:
        time.sleep(2)

    

    ngos = [campaign['ngo_name1'], campaign['ngo_name2'], campaign['ngo_name3'], 
            campaign['ngo_name4'], campaign['ngo_name5']]
    
    campaign_date = datetime.strptime(campaign["date"], "%Y-%m-%d")
    campaign_year = campaign_date.year

    # Filter out empty NGO names and join them with spaces
    ngos = "; ".join([ngo for ngo in ngos if ngo])
    
    # Modify the search query to focus on the NGO names and the company
    search_query = f"{campaign['company']} {ngos}".strip()  
    
    search_limit = 20  # Adjust to your needs
    
    # Perform the search (focuses on the finance subreddit)
    submissions = reddit.subreddit("finance").search(search_query, limit=search_limit)
    
    # Convert campaign date to timestamp range
    date_before = campaign_date + timedelta(days=365)
    date_after = campaign_date - timedelta(days=100)

    # Filter and collect data
    for submission in submissions:
        # Convert Reddit post date
        post_date = datetime.utcfromtimestamp(submission.created_utc)
        post_date = datetime.strptime(str(post_date), "%Y-%m-%d %H:%M:%S")
        post_date = post_date.strftime("%Y-%m-%d")
        post_date = datetime.strptime(post_date, "%Y-%m-%d")
        
        # Check if post falls within the date range
        if date_after <= post_date <= date_before:
            # Check if the NGO names are mentioned in the post's title or text
            if any(ngo.lower() in submission.title.lower() or ngo.lower() in submission.selftext.lower() for ngo in ngos.split("; ") or 'ngo' in submission.title.lower() or 'ngo' in submission.selftext.lower()):
                reddit_data.append({
                    "company": campaign["company"],
                    "NGO": ngos,
                    "date": campaign["date"],
                    "title": submission.title,
                    "text": submission.selftext,
                    "created_utc": submission.created_utc,
                    "url": submission.url,
                })
                

# Save to a DataFrame
df_reddit = pd.DataFrame(reddit_data)

# Save to a CSV file
df_reddit.to_csv("reddit_data.csv", index=False)



# Links Analysis

In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to check if the URL is accessible
def is_url_accessible(url, timeout=3):
    try:
        response = requests.head(url, timeout=timeout, allow_redirects=True)  # Allow redirects
        # Check for successful status codes (200, 301, 302)
        return response.status_code in [200, 301, 302]
    except requests.RequestException as e:
        print(f"Error checking URL {url}: {e}")
        return False

# Function to fetch and parse content from the final destination URL after redirection
def get_text_from_url(url, timeout=5, min_length=2700):
    try:
        # First make the GET request with redirection allowed
        response = requests.get(url, timeout=timeout, allow_redirects=True)
        
        # Check if there is a redirect
        if response.status_code in [301, 302]:
            final_url = response.headers['Location']  # Get the final redirect URL
            response = requests.get(final_url, timeout=timeout)  # Fetch the redirected URL

        # Skip non-HTML content types
        if 'text/html' not in response.headers.get('Content-Type', ''):
            return None

        # Check if the title contains '404' or 'page is missing' to ignore these pages
        try:
            soup = BeautifulSoup(response.text, 'lxml')
            title = soup.title.string if soup.title else ""
            if "404" in title.lower() or "error" in title.lower() or "ops" in title.lower():
                return None  # Skip this page
        except Exception:
            soup = BeautifulSoup(response.text, 'html5lib')
            title = soup.title.string if soup.title else ""
            if "404" in title.lower() or "error" in title.lower() or "ops" in title.lower():
                return None  # Skip this page

        # Extract and clean up text
        unwanted_chars = [',', '\n', '\t', ';', '\\']
        extracted_text = ' '.join(
            ''.join(char for char in p.get_text() if char not in unwanted_chars)
            for p in soup.find_all('p')
        )
        
        # Only save text if it is longer than the minimum length specified
        if len(extracted_text) < min_length:
            return None
        
        return extracted_text

    except (requests.RequestException, Exception):
        return None
    
# Sample DataFrame for testing (just 20 row)
#df_filtered = df.head(20)

# Process and fetch text for links
text_data = []

total_rows = len(df)

for index, row in df.iterrows():
    print(f"Processing row {index + 1}/{total_rows}...")
    
    texts = []
    for col in ['link1', 'link2', 'link3', 'link4', 'link5']:
        url = row[col]
        if pd.notna(url):  # Check if there is a URL
            text = get_text_from_url(url)
            if text:
                texts.append(text)
    combined_text = ' '.join(texts)
    text_data.append(combined_text)

    # Optionally, save text for each row in a file or handle it as needed
    if combined_text:  # Only save if there's non-null text
        file_name = f"text_files/text_data_row_{index + 1}.txt"
        with open(file_name, "w") as file:
            file.write(combined_text)
        #print(f"Saved text for row {index + 1} to {file_name}")

print("All rows processed and saved into separate text files.")


Processing row 1/6625...
Processing row 2/6625...
Processing row 3/6625...
Processing row 4/6625...
Processing row 5/6625...
Processing row 6/6625...
Processing row 7/6625...
Processing row 8/6625...
Processing row 9/6625...
Processing row 10/6625...
Processing row 11/6625...
Processing row 12/6625...
Processing row 13/6625...
Processing row 14/6625...
Processing row 15/6625...
Processing row 16/6625...
Processing row 17/6625...
Processing row 18/6625...
Processing row 19/6625...
Processing row 20/6625...
Processing row 21/6625...
Processing row 22/6625...
Processing row 23/6625...
Processing row 24/6625...
Processing row 25/6625...
Processing row 26/6625...
Processing row 27/6625...
Processing row 28/6625...
Processing row 29/6625...
Processing row 30/6625...
Processing row 31/6625...
Processing row 32/6625...
Processing row 33/6625...
Processing row 34/6625...
Processing row 35/6625...
Processing row 36/6625...
Processing row 37/6625...
Processing row 38/6625...
Processing row 39/662

/opt/anaconda3/envs/sim/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Processing row 104/6625...
Processing row 105/6625...
Processing row 106/6625...
Processing row 107/6625...
Processing row 108/6625...
Processing row 109/6625...
Processing row 110/6625...
Processing row 111/6625...
Processing row 112/6625...
Processing row 113/6625...
Processing row 114/6625...
Processing row 115/6625...
Processing row 116/6625...
Processing row 117/6625...
Processing row 118/6625...
Processing row 119/6625...
Processing row 120/6625...
Processing row 121/6625...
Processing row 122/6625...
Processing row 123/6625...
Processing row 124/6625...
Processing row 125/6625...
Processing row 126/6625...
Processing row 127/6625...
Processing row 128/6625...
Processing row 129/6625...
Processing row 130/6625...
Processing row 131/6625...
Processing row 132/6625...
Processing row 133/6625...
Processing row 134/6625...
Processing row 135/6625...
Processing row 136/6625...
Processing row 137/6625...
Processing row 138/6625...
Processing row 139/6625...
Processing row 140/6625...
P

In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to check if the URL is accessible
def is_url_accessible(url, timeout=2):
    try:
        response = requests.head(url, timeout=timeout, allow_redirects=True)  # Allow redirects
        # Check for successful status codes (200, 301, 302)
        return response.status_code in [200, 301, 302]
    except requests.RequestException as e:
        print(f"Error checking URL {url}: {e}")
        return False

# Function to fetch and parse content from the final destination URL after redirection
def get_text_from_url(url, timeout=2, min_length=2700):
    try:
        # First make the GET request with redirection allowed
        response = requests.get(url, timeout=timeout, allow_redirects=True)
        
        # Check if there is a redirect
        if response.status_code in [301, 302]:
            final_url = response.headers['Location']  # Get the final redirect URL
            response = requests.get(final_url, timeout=timeout)  # Fetch the redirected URL

        # Skip non-HTML content types
        if 'text/html' not in response.headers.get('Content-Type', ''):
            return None

        # Check if the title contains '404' or 'page is missing' to ignore these pages
        try:
            soup = BeautifulSoup(response.text, 'lxml')
            title = soup.title.string if soup.title else ""
            if "404" in title.lower() or "error" in title.lower() or "ops" in title.lower():
                return None  # Skip this page
        except Exception:
            soup = BeautifulSoup(response.text, 'html5lib')
            title = soup.title.string if soup.title else ""
            if "404" in title.lower() or "error" in title.lower() or "ops" in title.lower():
                return None  # Skip this page

        # Extract and clean up text
        unwanted_chars = [',', '\n', '\t', ';', '\\']
        extracted_text = ' '.join(
            ''.join(char for char in p.get_text() if char not in unwanted_chars)
            for p in soup.find_all('p')
        )
        
        # Only save text if it is longer than the minimum length specified
        if len(extracted_text) < min_length:
            return None
        
        return extracted_text

    except (requests.RequestException, Exception):
        return None
    
# Sample DataFrame for testing (just 20 row)
#df_filtered = df.head(20)

# Process and fetch text for links
text_data = []
df = df.reset_index(drop=True)
total_rows = len(df)

df = df.iloc[3315:]


for index, row in df.iterrows():
    print(f"Processing row {index + 1}/{total_rows}...")
    
    texts = []
    for col in ['link1', 'link2', 'link3', 'link4', 'link5']:
        url = row[col]
        if pd.notna(url):  # Check if there is a URL
            text = get_text_from_url(url)
            if text:
                texts.append(text)
    combined_text = ' '.join(texts)
    text_data.append(combined_text)

    # Optionally, save text for each row in a file or handle it as needed
    if combined_text:  # Only save if there's non-null text
        file_name = f"text_files/text_data_row_{index + 1}.txt"
        with open(file_name, "w") as file:
            file.write(combined_text)
        #print(f"Saved text for row {index + 1} to {file_name}")

print("All rows processed and saved into separate text files.")


Processing row 3316/6625...
Processing row 3317/6625...
Processing row 3318/6625...
Processing row 3319/6625...
Processing row 3320/6625...
Processing row 3321/6625...
Processing row 3322/6625...
Processing row 3323/6625...
Processing row 3324/6625...
Processing row 3325/6625...
Processing row 3326/6625...
Processing row 3327/6625...
Processing row 3328/6625...
Processing row 3329/6625...
Processing row 3330/6625...
Processing row 3331/6625...
Processing row 3332/6625...
Processing row 3333/6625...
Processing row 3334/6625...
Processing row 3335/6625...
Processing row 3336/6625...
Processing row 3337/6625...
Processing row 3338/6625...
Processing row 3339/6625...
Processing row 3340/6625...
Processing row 3341/6625...
Processing row 3342/6625...
Processing row 3343/6625...
Processing row 3344/6625...
Processing row 3345/6625...
Processing row 3346/6625...
Processing row 3347/6625...
Processing row 3348/6625...
Processing row 3349/6625...
Processing row 3350/6625...
Processing row 3351/